In [66]:
import theano
import theano.tensor as T
import theano.tensor.nnet as nnet
import numpy as np
import time

import cv2 
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import signal
import time
from math import sqrt
from skimage import transform
import skimage 


In [246]:
training_data = readTrainingdata('./training/train/img0_train0')

In [409]:
import random

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
#             print delta_nabla_b[0][0][0]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            if nabla_w[2][0][0] > 0:
                print nabla_w[2][0][0]
#                 print self.weights[0][0][0]-(eta * nabla_w[0][0][0])
        
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass

        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
                
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def update(self, x, y):
        
        nabla_b, nabla_w = self.backprop(x, y)        
        self.weights = [w - nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - nb for b, nb in zip(self.biases, nabla_b)]
         
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        """set a limit"""
        match_limit = 0.1
        output = sum(int(abs(x - y) < match_limit) for (x, y) in test_results)
#         print test_results[0][0] - test_results[0][1]
#         print test_results[10][0] - test_results[10][1]
#         print test_results[20][0] - test_results[20][1]        
        return output

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
    


In [410]:
def readTrainingdata(filename):
    
    trainingData = []
    theFile = open(filename, 'r')
    line = theFile.readline()
    while line != None and line != "":
        values = line.split()
        y = np.asarray([float(values[0])]).reshape(1,1)      
        x = np.asarray(values[1:], dtype=float).reshape(29,1)
        trainingData.append((x, y))
        line = theFile.readline()
    
    return trainingData

def getImgFromNet(net, img):
    
    width, height = img.shape
    filtered = np.zeros((width, height))

    radius = 3
    kernel = circle(radius)
    denominator = kernel.sum()
    img_force_int = img.astype('float')
    for x in range(radius, width-radius):
        for y in range(radius, height-radius):
            crop = img_force_int[x - radius:x + radius + 1, y - radius:y + radius + 1]
            croped_crop = crop[kernel == 1].reshape(29,1)
            output = net.feedforward(croped_crop)
            filtered[x, y] = output
    
    return filtered

In [411]:
test_data_batch = []
for i in range(0,5):    
    test_data_batch.append(training_data[random.randint(0,len(training_data)-1)])
    
net = Network([29, 50, 50, 1])

random.shuffle(training_data)
net.SGD(training_data[0:800], epochs=1, mini_batch_size=1, eta=100, test_data=test_data_batch)

0.0687245860514
2.63004717472e-42
1.48646101951e-48
2.97147942491e-50
1.5751291229e-48
2.04647799299e-48
1.78355250727e-50
6.08570047077e-49
1.6942027944e-49
6.19080566334e-48
9.96756579294e-49
6.90193598376e-49
3.29668525471e-51
8.42176693067e-50
1.34849829687e-49
5.66101450468e-49
1.27419048348e-45
2.30010876745e-47
1.29882708406e-47
5.8502690013e-48
4.69477027941e-50
1.07939831717e-48
1.27426012195e-48
1.26049788934e-45
5.87694781942e-47
5.46690426457e-49
2.5139845607e-49
3.24196677045e-47
2.27152751269e-48
1.99367696879e-48
1.57290497818e-49
2.36320685077e-48
1.07984389502e-47
1.01745308697e-48
6.34928040197e-47
3.95157054447e-47
1.02457830476e-48
1.03310752068e-46
7.24766727864e-49
4.15344353621e-48
5.26009886491e-45
6.12208687164e-49
3.06921924012e-48
2.50062986698e-51
1.6699213804e-52
1.24317857402e-48
3.77986972567e-47
8.77343543329e-49
4.68679379673e-43
3.03049414704e-49
9.48667464691e-51
2.30655441604e-48
1.31274866397e-44
4.24313988662e-48
3.55110465048e-50
5.02155284566e-47

In [416]:


weightsBefore = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsBefore
# print training_data[0][0][0]
random.shuffle(training_data)
# print training_data[0][0][0]
net.SGD(training_data[0:800], epochs=1, mini_batch_size=1, eta=10000, test_data=test_data_batch)

weightsAfter = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsAfter
for x, y in zip(weightsAfter, weightsBefore):
    print x - y

6.14768422469e-49
1.12434973461e-49
6.48626521757e-46
3.10013369477e-48
9.0371333609e-47
3.60583054997e-51
9.27981058608e-49
2.45137076811e-49
8.87619844064e-47
1.5924672899e-50
3.14825859503e-45
1.21453098945e-50
2.57167751959e-49
1.09371346646e-49
6.55031388822e-50
1.90084660264e-50
6.17721752861e-47
1.7602890004e-50
6.88032460032e-51
4.68193620012e-48
2.71091478606e-48
7.02540923408e-51
9.72724434082e-50
1.73459113631e-47
3.44540789829e-45
1.61547760464e-48
1.00334879958e-45
3.98058886604e-48
1.24205688343e-51
2.02230829325e-49
3.1492604222e-43
1.77280589062e-50
8.38211950229e-49
2.1949509658e-48
1.17841123815e-53
2.28930901202e-42
9.5079212971e-48
9.44219722405e-48
1.04717079693e-48
5.84547417213e-49
2.85646190926e-45
1.802724078e-51
5.52401561625e-45
4.95879198245e-49
3.00873057078e-48
2.84264898868e-48
8.15491062596e-51
5.13783408342e-48
1.16493113803e-47
2.06071019653e-50
2.95738790115e-48
2.57108264731e-47
5.15085512783e-51
3.64304250215e-50
4.05989531695e-49
1.74956124768e-47


In [314]:
weightsBefore = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsBefore
# random.shuffle(training_data)
net.SGD(training_data[0:800], epochs=1, mini_batch_size=100, eta=100000000000000000000, test_data=test_data_batch)

weightsAfter = [net.weights[2][0][0:5], net.weights[1][0][0:5], net.weights[0][0][0:5]]
# print weightsAfter
for x, y in zip(weightsAfter, weightsBefore):
    print x - y

4.13055481704e-51
-5.36830004879e-40
3.77681399086e-48
-1.77244112262e-38
2.98184939439e-53
1.00460855749e-45
1.40811799361e-49
2.48379639221e-54
-2.56888436417e-46
-6.37850389887e-49
-4.36558427653e-51
-8.98899677647e-47
7.12724660029e-54
-2.60014582676e-37
-1.98238509692e-53
-1.07293673974e-38
2.92725613951e-53
-5.1071733913e-42
1.16935939156e-49
3.63772723249e-47
5.00742278342e-45
-1.2141619781e-52
2.0346228361e-52
2.66222725538e-55
4.89356250369e-53
5.44521746429e-43
1.10786546049e-52
3.41810954149e-53
1.06462282363e-52
-6.53266064333e-53
-2.6420272378e-40
2.62819865264e-43
-2.52759076732e-46
2.88080445683e-54
-3.43867710242e-39
-9.65269795326e-45
5.97099216669e-48
1.46469655272e-40
1.81067013868e-52
1.96049410403e-51
6.95774762171e-52
-2.91262796329e-54
-3.29545812827e-40
8.00686839957e-52
1.32626237817e-44
2.46375732028e-50
8.04862309953e-49
-1.63166539496e-43
2.04771165387e-51
1.48976731303e-44
1.5358631917e-50
-3.68323401274e-49
-1.84929867139e-55
1.17320126288e-46
3.8562840196